In [ ]:
import mesa
import mesa_geo as mg
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from shapely.geometry import Polygon, MultiPolygon
import requests
import numpy as np

In [ ]:
# buurten ex water (approx 517)
url = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_BUURT_EXWATER&THEMA=gebiedsindeling"
ID = "Buurt"

# # woningswaardes 2022
price_url = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=WONINGWAARDE_2022&THEMA=woningwaarde"
price_ID = "LABEL"

r = requests.get(url)
geojson_ams = r.json()

r_price = requests.get(price_url)
geojson_ams_price = r_price.json()

In [ ]:
def create_gdf(data):
    geometries = []
    properties = []

    for feature in data:
        geom_type = feature['geometry']['type']
        coords = feature['geometry']['coordinates']
        
        if geom_type == 'Polygon':
            # Create a Polygon
            polygon = Polygon(shell=coords[0], holes=coords[1:] if len(coords) > 1 else None)
        elif geom_type == 'MultiPolygon':
            # Create a MultiPolygon
            polygons = []
            for polygon_coords in coords:
                polygon = Polygon(shell=polygon_coords[0], holes=polygon_coords[1:] if len(polygon_coords) > 1 else None)
                polygons.append(polygon)
            polygon = MultiPolygon(polygons)
        
        geometries.append(polygon)
        properties.append(feature['properties'])

    gdf = gpd.GeoDataFrame(properties, geometry=geometries)
    return gdf


data_spatial = geojson_ams['features']
data_price = geojson_ams_price['features']

gdf_spatial = create_gdf(data_spatial)
gdf_price = create_gdf(data_price)

# Join the two gdf files
nearest_gdf = gpd.sjoin_nearest(gdf_spatial, gdf_price, how="left", distance_col="distance")

# Update the properties of the first dataset with the matched LABEL values
for index, row in nearest_gdf.iterrows():
    if 'LABEL' in row:
        data_spatial[index]['properties']['price'] = row['LABEL']

geojson_ams['features'] = data_spatial


In [ ]:
class State(mg.GeoAgent):
    def __init__(self, unique_id, model, geometry, crs):
        super().__init__(unique_id, model, geometry, crs)


class GeoModel(mesa.Model):
    def __init__(self):
        self.space = mg.GeoSpace()

        ac = mg.AgentCreator(agent_class=State, model=self)
        agents = ac.from_GeoJSON(GeoJSON=geojson_ams, unique_id=ID)
        self.space.add_agents(agents)

m = GeoModel()

In [ ]:
geometries = []
agent_labels = []
price_ranges = [] 

for agent in m.space.agents:
    geometries.append(agent.geometry)
    price_ranges.append(agent.price)

cmap = plt.get_cmap('viridis') 
price_range_order = [
    '3012-3765',
    '3765-4518',
    '4518-5271',
    '5271-6024',
    '6024-6776',
    '6776-7529',
    '7529-9035',
    '9035-11294',
    '> 11294'
]
colors = {price_range: cmap(idx / len(price_range_order)) for idx, price_range in enumerate(price_range_order)}

gdf = gpd.GeoDataFrame(geometry=geometries)
gdf['price_range'] = price_ranges
gdf['color'] = gdf['price_range'].map(colors.get)


In [ ]:
def create_grid(gdf, cell_size):
    minx, miny, maxx, maxy = gdf.total_bounds

    # Generate the coordinates for the grid
    x_coords = np.arange(minx, maxx, cell_size)
    y_coords = np.arange(miny, maxy, cell_size)

    grid_cells = []
    for x in x_coords:
        for y in y_coords:
            # Create a cell
            cell = Polygon([(x, y), (x + cell_size, y), (x + cell_size, y + cell_size), (x, y + cell_size)])
            grid_cells.append(cell)

    # Create a GeoDataFrame from the grid cells
    grid_gdf = gpd.GeoDataFrame(grid_cells, columns=['geometry'])
    grid_gdf.crs = gdf.crs

    return grid_gdf

grid_gdf = create_grid(gdf, cell_size=800)

joined_gdf = gpd.sjoin(grid_gdf, gdf, how='left', predicate='intersects')

# Remove NaN rows
joined_gdf = joined_gdf.dropna(subset=['price_range'])

# Assign colors to each agent based on their price range
joined_gdf['color'] = joined_gdf['price_range'].map(colors.get)


In [ ]:
x_min = joined_gdf.total_bounds[0]
y_min = joined_gdf.total_bounds[1]

x_max = joined_gdf.total_bounds[2]
y_max = joined_gdf.total_bounds[3]

In [ ]:
scaled_joined_gdf = joined_gdf.geometry.copy()

scaled_joined_gdf = scaled_joined_gdf.translate(xoff=-x_min, yoff=-y_min)
scaled_joined_gdf = scaled_joined_gdf.scale(xfact=50/(x_max-x_min), yfact=50/(x_max-x_min), origin=(0, 0))

joined_gdf.geometry = scaled_joined_gdf

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10)) 
joined_gdf.plot(ax=ax, edgecolor='black', color=joined_gdf['color'], linewidth=0.5)

sm = ScalarMappable(cmap=cmap, norm=Normalize(vmin=0, vmax=len(price_range_order) - 1))
sm.set_array([]) 
cbar = fig.colorbar(sm, ax=ax, fraction=0.03) 
cbar.set_ticks(range(len(price_range_order)))
cbar.set_ticklabels(price_range_order)
cbar.set_label('Price Range')

geojson_str = joined_gdf.to_json()

with open('joined_gdf_50.geojson', 'w') as f:
    f.write(geojson_str)

plt.show()